Segmentación del mercado de adolecentes 
===


---

## Definición del problema real

Un vendedor desea enviar publicidad electrónica a una población de adolecentes y adultos jóvenes con el fin de maximizar sus ventas. Para ello, desea poder clasificar a sus clientes potenciales por grupos de interés de acuerdo con sus intereses y consecuentemente enviar publicidad específica a cada uno de ellos.   

## Definición del problema en términos de los datos

El problema consiste en poder determinar que grupos de interés existen en una población de clientes a partir de los mensajes enviados por un servicio de redes sociales. La información disponible consiste en 30000 observaciones de 40 variables que podrían caracterizar los intereses de la población analizada. Cada variable mide la frecuencia con que una determinada palabra aparece en los mensajes de texto; adicionalmente, dentro de estas variables se incluye  información como el sexo, la edad y la cantidad de contactos de la persona. 

## Exploración

In [0]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

APP_NAME = "Spark MLlib Application"

spark = SparkSession \
    .builder \
    .appName(APP_NAME) \
    .getOrCreate()

sc = spark.sparkContext

### Preprocesamiento del archivo original

In [0]:
##
## los datos contienen cabecera
##
!head files/snsdata.csv

gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,cute,sex,sexy,hot,kissed,dance,band,marching,music,rock,god,church,jesus,bible,hair,dress,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
2006,M,18.982,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,18.801,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2,1,0,0,0,6,4,0,1,0,0,0,0,0,0,0,0
2006,M,18.335,69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2006,F,18.875,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,NA,18.995,10,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5,1,1,0,3,0,1,0,0,0,1,0,0,0,2,0,0,0,0,0,1,1
2006,F,,142,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
2006,F,18.93,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0
2006,M,18.322,17,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,19.055

In [0]:
##
## extrae los nombres de las columnas
##
!head -n 1 files/snsdata.csv > names
!cat names

gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,cute,sex,sexy,hot,kissed,dance,band,marching,music,rock,god,church,jesus,bible,hair,dress,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs


In [0]:
##
## carga los nombres de las columas
##
names = open('names', 'r').read()
names = names.replace('\n', '')
names = names.split(',')
names

['gradyear',
 'gender',
 'age',
 'friends',
 'basketball',
 'football',
 'soccer',
 'softball',
 'volleyball',
 'swimming',
 'cheerleading',
 'baseball',
 'tennis',
 'sports',
 'cute',
 'sex',
 'sexy',
 'hot',
 'kissed',
 'dance',
 'band',
 'marching',
 'music',
 'rock',
 'god',
 'church',
 'jesus',
 'bible',
 'hair',
 'dress',
 'blonde',
 'mall',
 'shopping',
 'clothes',
 'hollister',
 'abercrombie',
 'die',
 'death',
 'drunk',
 'drugs']

In [0]:
##
## se crea un nuevo archivo sin la cabecera
##
!tail +2 files/snsdata.csv > snsdata
!head snsdata

2006,M,18.982,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,18.801,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2,1,0,0,0,6,4,0,1,0,0,0,0,0,0,0,0
2006,M,18.335,69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2006,F,18.875,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,NA,18.995,10,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5,1,1,0,3,0,1,0,0,0,1,0,0,0,2,0,0,0,0,0,1,1
2006,F,,142,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
2006,F,18.93,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0
2006,M,18.322,17,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,19.055,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,18.708,39,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,6,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0


In [0]:
##
## Carga de datos como un RDD
##   Todos los campos son strings
##
teens = sc.textFile("snsdata")
teens.collect()[0:10]

['2006,M,18.982,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '2006,F,18.801,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2,1,0,0,0,6,4,0,1,0,0,0,0,0,0,0,0',
 '2006,M,18.335,69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0',
 '2006,F,18.875,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '2006,NA,18.995,10,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5,1,1,0,3,0,1,0,0,0,1,0,0,0,2,0,0,0,0,0,1,1',
 '2006,F,,142,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0',
 '2006,F,18.93,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0',
 '2006,M,18.322,17,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '2006,F,19.055,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '2006,F,18.708,39,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,6,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0']

In [0]:
##
## Se reemplazan las cadenas de texto
##
teens = teens.map(lambda row: row.replace('M', '0'))
teens = teens.map(lambda row: row.replace('F', '1'))
teens = teens.map(lambda row: row.replace(',,', ',None,'))
teens = teens.map(lambda row: row.replace('NA', 'None'))
teens.collect()[0:10]

['2006,0,18.982,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '2006,1,18.801,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2,1,0,0,0,6,4,0,1,0,0,0,0,0,0,0,0',
 '2006,0,18.335,69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0',
 '2006,1,18.875,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '2006,None,18.995,10,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5,1,1,0,3,0,1,0,0,0,1,0,0,0,2,0,0,0,0,0,1,1',
 '2006,1,None,142,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0',
 '2006,1,18.93,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0',
 '2006,0,18.322,17,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '2006,1,19.055,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '2006,1,18.708,39,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,6,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0']

In [0]:
##
## Se convierten los strings a números
##
from numpy import array
teens = teens.map(lambda row: ([eval(x) for x in row.split(',')]))
print(teens.collect()[0:10])

[[2006, 0, 18.982, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2006, 1, 18.801, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 6, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [2006, 0, 18.335, 69, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [2006, 1, 18.875, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2006, None, 18.995, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 1, 1, 0, 3, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1], [2006, 1, None, 142, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0], [2006, 1, 18.93, 72, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0], [2006, 0, 18.322, 17, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0,

### Cómputo de estadísticos simples sobre el RDD para el género

In [0]:
##
## Cómputo del número de hombres y mujeres
## directamente sobre el RDD
## 0=M,1=F
##
print('Male = ', teens.filter(lambda row: row[1] == 0).count())
print('Female = ',teens.filter(lambda row: row[1] == 1).count())
print('None = ',teens.filter(lambda row: row[1] is None).count())

Male =  5222
Female =  22054
None =  2724


In [0]:
##
## Cómputo del número de hombres y mujeres
## usando SQL
##
df = teens.toDF(names)
df.createOrReplaceTempView('teens')
spark.sql('SELECT gender, COUNT(*) FROM teens GROUP BY gender').show()

+------+--------+
|gender|count(1)|
+------+--------+
|     0|    5222|
|  null|    2724|
|     1|   22054|
+------+--------+



### Cómputo de estadísticos simples sobre el RDD para la edad

In [0]:
##
## Estadisticas para la edad 
##   La muestra contiene un rango de edades 
##   por fuera de la población de interés
##

from pyspark.mllib.stat import Statistics

summary = Statistics.colStats(teens)
print('Minimo= ', summary.min()[2])
print('Maximo= ', summary.max()[2])
print('Promedio= ', summary.mean()[2])
print('Varianza= ', summary.variance()[2])
print('No ceros= ', summary.numNonzeros()[2])

Minimo=  3.086
Maximo=  106.927
Promedio=  nan
Varianza=  nan
No ceros=  30000.0


In [0]:
##
## Estadisticas para la edad 
##
sql = """
SELECT 
    min(age), 
    max(age), 
    mean(age),
    std(age)
FROM 
    teens 
"""
spark.sql(sql).show()

+--------+--------+------------------+------------------+
|min(age)|max(age)|          avg(age)|  stddev_samp(age)|
+--------+--------+------------------+------------------+
|   3.086| 106.927|17.995230902080486|7.8604164228228495|
+--------+--------+------------------+------------------+



### Imputación de datos

#### Cálculo de la edad promedio por año de graduación usando Map/Reduce

In [0]:
##
## Mapper: Genera las parejas (K, V) sin tener
## en cuenta los valores None
##
ages = teens.map(lambda x: (int(x[0]), (x[2], 1)) if x[2] is not None else None)
ages = ages.filter(lambda y: y is not None)
ages.collect()[1:20]

[(2006, (18.801, 1)),
 (2006, (18.335, 1)),
 (2006, (18.875, 1)),
 (2006, (18.995, 1)),
 (2006, (18.93, 1)),
 (2006, (18.322, 1)),
 (2006, (19.055, 1)),
 (2006, (18.708, 1)),
 (2006, (18.543, 1)),
 (2006, (19.463, 1)),
 (2006, (18.097, 1)),
 (2006, (18.398, 1)),
 (2006, (18.987, 1)),
 (2006, (17.158, 1)),
 (2006, (18.497, 1)),
 (2006, (18.738, 1)),
 (2006, (19.296, 1)),
 (2006, (18.752, 1)),
 (2006, (19.036, 1))]

In [0]:
##
## Reducer: cálcula la suma de edades y la 
## cantidad de datos
##
ages = ages.reduceByKey(lambda a, b: (a[0] + b[0], a[1]+b[1]))
ages.collect()

[(2006, (120660.3039999998, 6305)),
 (2008, (108875.78400000033, 6213)),
 (2007, (116142.06500000008, 6315)),
 (2009, (102623.10599999984, 6081))]

In [0]:
##
## Mapper: calcula la edad promedio por
## año de graduación
##
ages = ages.map(lambda y: [y[0], y[1][0] / y[1][1]])
ages.collect()

[[2006, 19.137240919904805],
 [2008, 17.523866731047857],
 [2007, 18.39145922406969],
 [2009, 16.876024666995534]]

In [0]:
##
## Convierte la lista de listas
## en un diccionario
##
mean_age = {}
for age in ages.collect():
    mean_age[age[0]] = age[1]
mean_age

{2006: 19.137240919904805,
 2008: 17.523866731047857,
 2007: 18.39145922406969,
 2009: 16.876024666995534}

In [0]:
##
## reemplaza los NA en edad por el promedio
##
def f(row):
    row[2] = mean_age[row[0]] if row[2] is None else row[2] 
    return row
    
teens = teens.map(lambda row: f(row))

In [0]:
##
## filtra por edades
##
teens = teens.filter(lambda row: row[2] >= 13 and row[2] < 20)
print(teens.collect()[0:5])
teens.count()

[[2006, 0, 18.982, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2006, 1, 18.801, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 6, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [2006, 0, 18.335, 69, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [2006, 1, 18.875, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2006, None, 18.995, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 1, 1, 0, 3, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1]]


29563

#### Cálculo de la edad promedio por año de graduación usando SQL

In [0]:
##
## Calcula el promedio de edad por año 
## de graduación usando SQL
## 
sql = """
SELECT 
    gradyear, 
    mean(age) 
FROM 
    teens
WHERE 
    age IS NOT NULL
GROUP BY 
    gradyear 
"""
df_ages = spark.sql(sql)
df_ages.show()

+--------+------------------+
|gradyear|          avg(age)|
+--------+------------------+
|    2007| 18.39145922406969|
|    2009|16.878190900098883|
|    2006|19.137240919904805|
|    2008|17.524112041210614|
+--------+------------------+



In [0]:
##
## Convierte el DataFrame en un RDD
##
df_ages.rdd.collect()

[Row(gradyear=2007, avg(age)=18.39145922406969),
 Row(gradyear=2009, avg(age)=16.878190900098883),
 Row(gradyear=2006, avg(age)=19.137240919904805),
 Row(gradyear=2008, avg(age)=17.524112041210614)]

In [0]:
## 
## Se puede acceder a los elementos de forma normal
## 
for row in df_ages.rdd.collect():
    print(row[0], row[1])
    
##
## se debe usar el mismo codigo de actualiación
## de la sección anterior, ya que SparkQL no
## soporta el comando UPDATE
##

2007 18.39145922406969
2009 16.878190900098883
2006 19.137240919904805
2008 17.524112041210614


# Entrenamiento del modelo

In [0]:
##
## Selecciona las columnas asociadas a los
## intereses de la población analizada
##
interests = teens.map(lambda row: row[4:])

In [0]:
##
## Importa la librería KMeans
## 
from pyspark.mllib.clustering import KMeans, KMeansModel

In [0]:
##
## Se usa el algoritmo para determinar los centros de 5 grupos
##
model = KMeans.train(rdd=interests, 
                     k=5, 
                     maxIterations=1000, 
                     initializationMode="random")

# Evaluación del modelo

In [0]:
##
## centros de los clusters
##
model.clusterCenters

[array([0.35953177, 0.37959866, 0.24414716, 0.19230769, 0.19565217,
        0.159699  , 0.18979933, 0.08779264, 0.10117057, 0.1270903 ,
        0.68394649, 0.1638796 , 0.22993311, 0.22491639, 0.09531773,
        4.61789298, 0.34531773, 0.03929766, 0.92976589, 0.33528428,
        0.51839465, 0.4506689 , 0.11036789, 0.02759197, 0.61538462,
        0.30685619, 0.09782609, 0.46153846, 0.64799331, 0.23494983,
        0.10033445, 0.09113712, 0.24414716, 0.13795987, 0.11120401,
        0.05518395]),
 array([0.23190993, 0.21735549, 0.1973088 , 0.14600211, 0.13034922,
        0.11336903, 0.09616001, 0.09167468, 0.07478603, 0.11400979,
        0.25790654, 0.09698384, 0.11689322, 0.10503913, 0.04906403,
        0.20151952, 0.10178956, 0.01487482, 0.30559751, 0.1489313 ,
        0.28495583, 0.17996247, 0.07588448, 0.01244908, 0.2357545 ,
        0.08279555, 0.05286283, 0.20682869, 0.29973912, 0.1058172 ,
        0.05931622, 0.04178681, 0.13304957, 0.08705204, 0.0641677 ,
        0.03190077]),
 arr

In [0]:
##
## calcula el indice del cluster al que 
## pertenece cada ejemplo
##
clusters = model.predict(interests)
clusters.collect()[:20]

[1, 2, 4, 1, 4, 4, 1, 1, 1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1]

In [0]:
##
## Número de ejemplos en cada cluster
##
import collections
collections.Counter(clusters.collect())

Counter({1: 21849, 2: 1131, 4: 4781, 3: 606, 0: 1196})

# Análisis del modelo

In [0]:
##
## se agrega el cluster a los datos originales
##
import numpy as np
teens.collect()[0:4]
teens_aug = teens.map(lambda row: [model.predict(row[4:])] + row)

In [0]:
##
## Crea un DF para analizar los resultados 
## mas facilmente
##
resultDF = teens_aug.toDF(['cluster'] + names)
resultDF.createOrReplaceTempView('result')

In [0]:
##
## clusters a los que pertenecen los primeros cinco patrones
##
spark.sql('SELECT cluster, gender, age, friends FROM result LIMIT 5').show()

+-------+------+------+-------+
|cluster|gender|   age|friends|
+-------+------+------+-------+
|      1|     0|18.982|      7|
|      2|     1|18.801|      0|
|      4|     0|18.335|     69|
|      1|     1|18.875|      0|
|      4|  null|18.995|     10|
+-------+------+------+-------+



In [0]:
##
## Características demográficas de los clusters
##
sql = """
SELECT
    cluster,
    mean(age),
    mean(friends)
FROM
    result
GROUP BY
    cluster
"""
spark.sql(sql).show()

+-------+------------------+-----------------+
|cluster|          avg(age)|     avg(friends)|
+-------+------------------+-----------------+
|      0| 17.15160721326572|37.59364548494983|
|      1|17.395222576583617|29.27406288617328|
|      3|17.454890111921852|35.08910891089109|
|      2|17.135859726985856|31.32449160035367|
|      4|17.346373681232055|32.07759882869693|
+-------+------------------+-----------------+

